In [1]:
import requests
import bs4
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pickle
import joblib

sw = stopwords.words("russian")

# Ссылка на сайт с которого берем отзывы

In [46]:
link = 'https://torg.mail.ru/review/goods/mobilephones/'

# Собственно собираем текста отзывов вместе с оценкой

In [47]:
texts = []
scores = []
for i in range(1,300):
    req = requests.get(link+"?page="+str(i))
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    for rev in parser.find_all('div', attrs={'class':'review-item__body'}):
        sub_texts = str('')
        for text in rev.find_all('p', attrs={'class':'review-item__paragraph'}):
            sub_texts = sub_texts + "\n" + text.find('span').text
        texts.append(sub_texts)
        scores.append(rev.find('span', attrs={'class':'review-item__rating-counter'}).text)

In [48]:
print scores[5]
print texts[5]

1

Разочарован полностью ! После 2 месяцев бережного использования ( телефон был в чехле , с наклеенной защитной пленкой и носился только в нагрудном кармане , не ронялся не контактировал с водой ) телефон стал ужасно греться и у телефона повело корпус !!!! и лопнул дисплей !!!! ( " легендарное"... 
Стильный , довольно шустрый , хороший аудио плеер , есть радио и nfc												
Качество сборки и комплектующих.												


In [49]:
print len(scores)
print len(texts)

5980
5980


# Сохраняем все в дата фрейм и затем в csv

In [50]:
df = pd.DataFrame()

In [51]:
df['texts'] = texts
df['scores'] = scores

In [118]:
df.to_csv('data.csv', encoding='utf-8', index=False)

## Прочитаем то что записали и немного поколдуем для удаления всяких ненужных символов

In [2]:
new_df = pd.read_csv('data.csv')

In [3]:
new_df['texts'] = new_df['texts'].apply(lambda x: x.replace('\t', ''))
new_df['texts'] = new_df['texts'].apply(lambda x: x.replace('\n', ''))
new_df['scores'] = new_df['scores'].apply(lambda x: x.replace(',', '.'))
new_df['scores'] = new_df['scores'].apply(lambda x: 0 if (float(x) <= 3) else 1)

# Считываем тестовый файл

In [12]:
with open('test.csv') as f:
    data = f.read()
test_revs = [r.text for r in bs4.BeautifulSoup(data, 'lxml').find_all('review')]

# Подбираем модель

In [18]:
vect = TfidfVectorizer(stop_words=sw)
model = LinearSVC()
#model = LogisticRegression()
pipe = Pipeline([('vectorizer', vect), ('classifier', model)])

In [16]:
params_grid = {'vectorizer__ngram_range':[(1,1),(1,2),(1,3),(1,4)], 'classifier__C':[0.01, 0.1, 1, 10]}

In [19]:
grid = GridSearchCV(pipe, params_grid, scoring='accuracy')

In [20]:
grid.fit(new_df.texts, new_df.scores)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)], 'classifier__C': [0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [21]:
grid.best_params_

{'classifier__C': 10, 'vectorizer__ngram_range': (1, 2)}

In [22]:
res = grid.best_estimator_.fit(new_df.texts, new_df.scores)

# Пишем результат в файл

In [15]:
out = pd.DataFrame(grid.best_estimator_.predict(test_revs), columns=['y'])
out.index.name = 'Id'
out['y'] = out['y'].apply(lambda x: 'neg' if (x==0) else 'pos')
out.to_csv('submission.csv')

In [119]:
vect = TfidfVectorizer(stop_words=sw, ngram_range=(1,2))
model = LinearSVC(C=10)

In [121]:
vectorized = vect.fit_transform(new_df.texts)
model.fit(vectorized, new_df.scores)

LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [122]:
with open('model.pkl', 'w') as m:
    pickle.dump(model, m)
    
with open('vectorizer.pkl', 'w') as v:
    pickle.dump(vect, v)

In [126]:
model.predict(vect.transform(test_revs))

array([0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1])

In [37]:
vect = TfidfVectorizer(stop_words=sw, ngram_range=(1,2))
model = LinearSVC(C=10)

In [38]:
vect.fit(new_df.texts)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=[u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u...043a\u043e\u043d\u0435\u0447\u043d\u043e', u'\u0432\u0441\u044e', u'\u043c\u0435\u0436\u0434\u0443'],
        strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [39]:
model.fit(vectorized, new_df.scores)

LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [33]:
res.predict(vect.transform(["плохо", "хорошо"]))[0]

0

In [201]:
res.predict(["This is the great bank, just awesome"])[0]

1

In [114]:
class SentimentClassifier():
    def __init__(self, model, vect):
        self.model = model
        self.vectorizer = vect
        self.classes_dict = {0: "negative", 1: "positive", -1: "prediction error"}
        

    def predict_text(self, text):
        try:
            vectorized = self.vectorizer.transform([text])
            return self.model.predict(vectorized)[0]
        except:
            print "prediction error"
            return -1
        
    def predict_list(self, list_of_texts):
        try:
            vectorized = self.vectorizer.transform(list_of_texts)
            return self.model.predict(vectorized)
        except:
            print 'prediction error'
            return None

    def get_prediction_message(self, text):
        class_prediction = self.predict_text(text)
        return self.classes_dict[class_prediction]
    
    def get_prediction_message(self, list_of_texts):
        class_prediction = self.predict_list(list_of_texts)
        for el in class_prediction:
            print self.classes_dict[el]
        return 

In [115]:
clf = SentimentClassifier(model, vect)

#pred = clf.get_prediction_message(["This is the great bank, just awesome"])
#print pred

In [58]:
clf.get_model()

LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [59]:
clf.get_vectorizer()

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=[u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u...043a\u043e\u043d\u0435\u0447\u043d\u043e', u'\u0432\u0441\u044e', u'\u043c\u0435\u0436\u0434\u0443'],
        strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [118]:
clf.get_prediction_message(["This is the great bank, just awesome", "плохо"])

positive
negative


In [90]:
clf.predict_list()

prediction error
